<a href="https://colab.research.google.com/github/sungkim11/TaxGPT/blob/main/Create_IRC_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Python Packages

In [ ]:
%%writefile requirements.txt
openai
chromadb
langchain
tiktoken
unstructured

Writing requirements.txt


In [ ]:
%pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 KB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.6/425.6 KB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing 

Import Python Packages

In [ ]:
import os
import platform

import openai
import chromadb
import langchain

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import TokenTextSplitter
from langchain.llms import OpenAI
from langchain.document_loaders import UnstructuredURLLoader

import urllib
from urllib import request

print('Python: ', platform.python_version())

Python:  3.9.16


Mount Storage - Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Set OpenAI API Key

In [ ]:
os.environ["OPENAI_API_KEY"] = "openai api key"

List Bloomberg Tax Internal Revenue Codes URLs

In [ ]:
urls = [
   "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-65/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-50",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4907",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_1411",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-61/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-66/subchapter-d",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-42/subchapter-e",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-77",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7124",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7701",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_3306",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_9007",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-32/subchapter-d",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-32/subchapter-g",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7524",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-1/subchapter-g",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_3501",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_6903",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-70/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7525",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-43",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_3508",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-i/chapter-98/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-e/chapter-51/subchapter-c",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4974",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4904",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7519",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_9004",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_9005",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4978b",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-e/chapter-53",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_2702",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-78/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-31/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_3511",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_9034",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-i/chapter-98/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-c/chapter-23a",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_9036",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4905",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-1/subchapter-j",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7501",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-33/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_9035",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-75/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-6",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-44",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-k",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-1/subchapter-v",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-5",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-c/chapter-21/subchapter-d",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4980g",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-76/subchapter-e",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-b/chapter-12/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-h/chapter-95",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-6/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-3",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-80/subchapter-c",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_5000b",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4976",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-h/chapter-96",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-35",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-2",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4980c",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7518",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-e/chapter-52/subchapter-f",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7103",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-c/chapter-21/subchapter-c",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7505",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_6807",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-42/subchapter-h%E2%80%94excise",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-c/chapter-21/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7506",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_3309",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_8021",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-64/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-b/chapter-11/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_3305",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4981",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-64/subchapter-d",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-e/chapter-52/subchapter-e",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_2801",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-68/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-b/chapter-12",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_1403",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-69",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-e/chapter-52/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-e/chapter-51/subchapter-f",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-j",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_9033",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7101",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-c/chapter-22/subchapter-e",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-e/chapter-51",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7527",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7523",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-b/chapter-11/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4502",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_6801",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4902",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-e/chapter-51/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7516",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7702",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-50a%E2%80%94designated",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_5891",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-c/chapter-22/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-79",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_3505",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-33/subchapter-c",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7702b",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-62/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_9008",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-j/chapter-99/subchapter-c",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_9012",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-j/chapter-99/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-46",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-31",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-c/chapter-21/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-35/subchapter-c",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_3303",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_6905",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_9040",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-75/subchapter-c",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4980f",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-72/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7526",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7102",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-1/subchapter-t",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_1471",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_3507",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-1/subchapter-o",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_2704",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-32/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7514",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-e/chapter-51/subchapter-i",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-j/chapter-99/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-e",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-63/subchapter-d",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-1/subchapter-u",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-61/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_6805",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_5000",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_8001",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-b/chapter-15",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_6904",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-47",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-76/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-67/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-c/chapter-22/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-e/chapter-51/subchapter-h",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-63/subchapter-c",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4980e",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_2701",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_1491",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_8002",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-1/subchapter-r",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4985",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_1473",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-42",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4980i",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4980d",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-1",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-42/subchapter-f",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-c/chapter-24",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-66/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-b/chapter-13",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_1402",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_9032",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-e/chapter-53/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-e/chapter-51/subchapter-g",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_9041",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-37",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4999",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-g",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-c",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-34/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4906",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-48",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-1/subchapter-f",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-j/chapter-99/subchapter-d",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-71",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-73",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-c/chapter-22/subchapter-d",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_6803",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-b/chapter-13/subchapter-d",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-b/chapter-13/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4912",
    "https://irc.bloombergtax.com/public/uscode/toc/irc",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-e/chapter-51/subchapter-d",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-1/subchapter-m",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4978a",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_8003",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-66/subchapter-c",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4977",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_9003",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4701",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-74",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-1/subchapter-z",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-e/chapter-52/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7527a",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-36/subchapter-d",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-e/chapter-53/subchapter-c",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_9039",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-c/chapter-22/subchapter-c",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-e/chapter-55",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-k/chapter-100/subchapter-c",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7123",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-e/chapter-54",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-1/subchapter-e",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-e/chapter-52/subchapter-c",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-c/chapter-23",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_6804",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-63/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-e/chapter-51/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-b/chapter-12/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-45",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7512",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-64/subchapter-c",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-68",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_1401",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-40",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-75",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-b/chapter-13/subchapter-f",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-36",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-66",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-1/subchapter-i",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-k/chapter-100/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-42/subchapter-c",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-c/chapter-24/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_3308",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_3502",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-38/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_9037",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-39",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_3304",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-75/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-1/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-1/subchapter-y",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4980b",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-1/subchapter-k",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-e/chapter-52",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_1493",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7526a",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-72",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-g/chapter-92",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-b/chapter-13/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-42/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4972",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-c/chapter-22",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-b/chapter-12/subchapter-c",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_3307",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_8023",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_9002",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-32/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-e/chapter-53/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_9042",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-38/subchapter-c",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_2703",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-6/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-38",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-k/chapter-100/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_3321",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-32/subchapter-e",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7529",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-80/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-1/subchapter-x",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7703",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-80/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-67/subchapter-d",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-38/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_9500",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_6901",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-b/chapter-14",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7502",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-78/subchapter-d",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_8004",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-76",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_9031",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7508a",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-64/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_9009",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-32",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4980",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-1/subchapter-p",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-e/chapter-51/subchapter-e",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-62",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-38/subchapter-d",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_9001",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7517",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7522",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-32/subchapter-c",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7503",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-1/subchapter-q",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7528",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7530",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7121",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-36/subchapter-f",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_3509",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-i",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-65",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-36/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4901",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4979a",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-c/chapter-25",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_9006",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-70",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-33",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-78",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-b/chapter-13/subchapter-g",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4501",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_3510",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-41",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_6902",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-64",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7510",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-b",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_9010",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-g/chapter-91",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-2a",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_8022",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_9013",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-76/subchapter-c",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7509",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-1/subchapter-h",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-36/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_3301",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-78/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-e/chapter-51/subchapter-j",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-1/subchapter-c",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-63",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_8005",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-68/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_9038",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-b/chapter-11/subchapter-c",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4973",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-b/chapter-11",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_3310",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_1492",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4975",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-1/subchapter-w",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-32/subchapter-f",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-49",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7504",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-42/subchapter-g",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-e/chapter-53/subchapter-d",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-66/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_5000a",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_3504",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-70/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-62/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_9011",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7507",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-67",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-1/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_1474",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-42/subchapter-d",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4978",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7513",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_5000c",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4982",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-1/subchapter-n",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_3506",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-j/chapter-99",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-3/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-67/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-b/chapter-13/subchapter-c",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-1/subchapter-d",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-35/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-31/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-75/subchapter-d",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-1/subchapter-l",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-3/subchapter-b",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_5000d",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-e/chapter-52/subchapter-g",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-63/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-31/subchapter-c",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-42/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-65/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4971",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_3512",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_3311",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4503",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-76/subchapter-d",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_1472",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4911",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7508",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-76/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_3302",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-b/chapter-13/subchapter-e",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-h",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-72/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-35/subchapter-a",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4980h",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7521",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-1/subchapter-s",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_5881",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-i/chapter-98",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7520",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-67/subchapter-c",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_1494",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_6808",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-80",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4980a",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_3503",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7705",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4979",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-68/subchapter-c",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-61",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-f/chapter-64/subchapter-e",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-33/subchapter-e",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_6802",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-c/chapter-21",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-e/chapter-52/subchapter-d",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-k/chapter-100",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7122",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-a/chapter-4",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_3322",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7704",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-34",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_4903",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_6806",
    "https://irc.bloombergtax.com/public/uscode/doc/irc/section_7702a",
    "https://irc.bloombergtax.com/public/uscode/toc/irc/subtitle-d/chapter-34/subchapter-b"
]


Load Bloomberg Tax Internal Revenue Codes

In [ ]:
loader = UnstructuredURLLoader(urls=urls)

In [ ]:
irc_data = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
collection_name="irc"
persist_directory="/content/drive/My Drive/Colab Notebooks/chromadb/irc"

In [ ]:
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=0)
irc_doc = text_splitter.split_documents(irc_data)

In [ ]:
embeddings = OpenAIEmbeddings()
irc_db = Chroma.from_documents(irc_doc, embeddings, collection_name=collection_name, persist_directory=persist_directory)
irc_db.persist()

DEBUG:Chroma:Logger created


Running Chroma using direct local API.
No existing DB found in /content/drive/My Drive/Colab Notebooks/chromadb/irc, skipping load
No existing DB found in /content/drive/My Drive/Colab Notebooks/chromadb/irc, skipping load


DEBUG:Chroma:Index not found
DEBUG:Chroma:Index saved to /content/drive/My Drive/Colab Notebooks/chromadb/irc/index/index.bin
DEBUG:Chroma:Index saved to /content/drive/My Drive/Colab Notebooks/chromadb/irc/index/index.bin


Persisting DB to disk, putting it in the save folder /content/drive/My Drive/Colab Notebooks/chromadb/irc
